In [8]:
import os
import sys
import json
import math
import numpy as np
from glob import glob
from PIL import Image
import nibabel as nib

import torch
from torch import nn
from torch.utils import data
import torch.multiprocessing as mp

from pytorch_lightning import Trainer
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import ModelCheckpoint


sys.path.insert(0, "../../cornucopia")
import cornucopia as cc

os.environ['PYTORCH_JIT_USE_NNC_NOT_NVFUSER'] = '1'
torch.set_float32_matmul_precision('medium')


In [6]:
class RealData(object):
    def __init__(self):
        #super().__init__(self)

        self.nifti_dir = {"oct": "/autofs/cluster/octdata2/users/Chao/caa/caa_17/occipital/process_run1/dBI_vessel/nii_split/I_mosaic_1_1_0.mgz",
                    "prediction": "/autofs/cluster/octdata2/users/epc28/veritas/output/real_data/smoothed_prediction.nii.gz"}
        
        self.tiff_dir = {"oct": "/autofs/cluster/octdata2/users/epc28/veritas/output/real_data/tiffs/oct_tiffs",
                         "prediction": "/autofs/cluster/octdata2/users/epc28/veritas/output/real_data/tiffs/prediction_tiffs"}
        
        self.oct_or_pred = 'oct'
        self.dtype = "float32"

        self.tensor = cc.LoadTransform(device='cpu', to_ras=False)(self.nifti_dir[self.oct_or_pred])[None]
        self.arr = np.array(self.tensor, dtype=self.dtype)[0][0]
    

    def makeTiffs(self):
        nifti_used = self.nifti_dir[self.oct_or_pred]
        tiff_used = self.tiff_dir
        self.tensor = cc.LoadTransform(device='cpu', dtype=self.dtype, to_ras=False)(self.nifti_dir['oct'])[None]


    def outputDirChecker(self, p):
        if os.path.exists(p):
            [os.remove(x) for x in glob(f'{p}/*')]
        else:
            os.mkdir(p)


RD = RealData()

In [27]:
t = RD.tensor[0][0][:][:][128]
t = t.numpy()
t = t / t.max()

In [29]:
img = Image.fromarray(t).convert("RGB") * 256
img.save('/autofs/cluster/octdata2/users/epc28/veritas/output/real_data/tiffs/test.tiff')